# 개인프로젝트
###  #01.프로젝트 개요

## 관심사
- 관심있는 브랜드의 남성 스포츠 용품에 대한 정보 수집 : 미즈노, 아디다스 


        

        아디다스 남성 스포츠 용품 웹페이지 url : https://www.adidas.co.kr/men-performance?start=0


- 미즈노 크롤링 하나 데이터프레임(완료)+ 아디다스 크롤링 데이터 프레임 하나로  합치기 


In [1]:
# 필요한 모듈 참조
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame    

import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time


#### #02.웹 페이지 코드 수집 - 아디다스 


미즈노는 각 페이지 별로 상품 수집해서 그 전에는 이미지링크 + 상품 상세보기 링크 + 상품이름 -> 링크를 타고 들어가자 
그다음에 정가, 할인가 + 리뷰 수 

In [2]:

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(5)

In [3]:

## https://www.adidas.co.kr/men-performance?start=0
## https://www.adidas.co.kr/men-performance?start=48

## 페이지마다 url 값이 48씩 늘어남
adidas_num = []
for i in range(0,25) :
    adidas_num.append(i*48)
print(adidas_num)



[0, 48, 96, 144, 192, 240, 288, 336, 384, 432, 480, 528, 576, 624, 672, 720, 768, 816, 864, 912, 960, 1008, 1056, 1104, 1152]


#### #03. 응답결과로부터 내용 추출하기

In [63]:

## 아디다스 빈 list
adidas_list =[]
# ## 페이지 수 만큼 반복  25페이지 까지 있음
# ## 수집할 컨텐츠가 있는 웹페이지의 주소 
# url = "https://www.adidas.co.kr/men-performance?start=0" 
    
#이미지링크 + 상품 상세보기 링크 + 상품이름
url = "https://www.adidas.co.kr/men-performance?start=0"

driver.get(url)
time.sleep(1)
## 전체 HTML
soup = BeautifulSoup(driver.page_source)
adidas_list = soup.select(".grid-item")
print("상품의 수 : ", len(adidas_list))
result_dict = {}
for i, adidas in enumerate(adidas_list):
    print("-" * 50)
    print("%d번째 상품" % i)
    # 상품이름
    adidas_nameEl = adidas.select(".glass-product-card__title")
    adidas_name = adidas_nameEl[0].text.strip()
    # print(adidas_name)

    ## 이미지 링크 
    # https://assets.adidas.com/images/w_383,h_383,f_auto,q_auto,fl_lossy,c_fill,g_auto/9beea668a3034150a538ac8300b51db6_9366/m-pl-티.jpg

    adidas_imageEl = adidas.select(".glass-product-card__assets > a > img")
    # print(adidas_imageEl)
    adidas_image = adidas_imageEl[0].attrs['src']
    # print(adidas_image)
    
    ## 상세보기 링크 

    adidas_detailEl = adidas.select(".glass-product-card > a ")
    adidas_detail = adidas_detailEl[0].attrs['href']
    if adidas_detail.find("https://www.adidas.co.kr") == -1 :
        adidas_detail = "https://www.adidas.co.kr" + adidas_detail

    # print(adidas_detail)

    ## 상세보기 페이지 들어가기 전에 가격이랑 할인가 할인금액=none 놓고 + 솔드아웃 표시
    ## if 써서 리뷰 태그 있으면 리뷰태그 , 없으면 none입니다 ㄱㄱ 
    adidas_priceEl = adidas.select(".gl-price .gl-price-item ")
    # print(len(adidas_priceEl)) ## -> 비어있으면 soldout 임 
    if len(adidas_priceEl) == 2 :
        adidas_BasePrice = adidas_priceEl[0].text.strip()
        adidas_LastPrice = adidas_priceEl[1].text.strip()
        adidas_discountPrice = "-"
    elif len(adidas_priceEl) == 1:
        adidas_BasePrice = adidas_priceEl[0].text.strip()
        adidas_LastPrice = adidas_BasePrice
        adidas_discountPrice = "-"
    else :
        adidas_BasePrice = "재고없음"
        adidas_LastPrice = "재고없음"
        adidas_discountPrice = "-"

    # print(adidas_BasePrice,adidas_LastPrice)
    ## 상세페이지 들어가서 리뷰개수 가져오기 ㄱㄱ 
     # ---------------------------------------------------
            
    #### 상세보기 수집 (시작) 

    # --------------mizuno_detail-------------------------------------
            
    driver.get(adidas_detail)
    time.sleep(1)
    ## 상세보기 HTML            
    detailSoup = BeautifulSoup(driver.page_source)
    # print(detailSoup)

    # 정가, 할인금액,최종금액 + 리뷰 수 추출
    reviewEl = detailSoup.select(".reviews-header___1kFSZ h2")
    # print(reviewEl)   
    if len(reviewEl) ==1 :
        review = reviewEl[0].text.strip()
    else : 
        review = "리뷰없음"
    # print(review)

            
            
    # ---------------------------------------------------
            
    #### 상세보기 수집 (끝) 

    # --------------mizuno_detail-------------------------------------



        

    
        

    

    ## 솔드아웃 셀렉트 하고 솔드아웃이면 정가,할인가,리뷰 ...
    ## 품절입니다 처리 
    ## else 
    ## 상세보기 링크 들어가기 





상품의 수 :  48
--------------------------------------------------
0번째 상품
리뷰 (29)
--------------------------------------------------
1번째 상품
리뷰 (18)
--------------------------------------------------
2번째 상품
리뷰 (3)
--------------------------------------------------
3번째 상품
리뷰 (9)
--------------------------------------------------
4번째 상품
리뷰없음
--------------------------------------------------
5번째 상품
리뷰 (7)
--------------------------------------------------
6번째 상품


KeyboardInterrupt: 

In [ ]:
# 데이터 형식 \통일 + 엑셀파일 ㄱㄱ 


NameError: name 'df' is not defined